In [1]:
from jw_package import *
from functools import reduce

import configuration completed !
train + key + weather merging started
train + key + weather merging finished
function configuration completed ! 
Good to go !


In [22]:
results =[]

for each in range(1,21):
    file_path = '../data/weather_mean/{:02}.csv'.format(each)
    results.append(pd.read_csv(file_path,index_col=0))

mean = reduce((lambda x,y: x.append(y)),results)

In [24]:
sales = pd.read_csv('../data/basic/train.csv')
keys = pd.read_csv('../data/basic/key.csv')

In [25]:
data = sales.merge(keys).merge(mean)

In [41]:
cols = list(data.columns[3:])
cols.remove('station_nbr')
cols.remove('sunset')
cols.remove('sunrise')
cols.remove('codesum')
cols.remove('snowfall')
cols.remove('day')
cols.remove('month')
cols.remove('year')

In [42]:
formula = columns_to_relation(cols,None,'units',False)

In [43]:
formula

'units ~ avgspeed + cool + depart + dewpoint + heat + preciptotal + resultdir + resultspeed + sealevel + stnpressure + tavg + tmax + tmin + wetbulb'

In [44]:
results=[0,]
mean = 0
n = 0
for item_nbr in range(1,112):
    frame = data[data['item_nbr']==item_nbr]
    
    model = sm.OLS.from_formula(formula,data=frame)
    
    r2 = model.fit().rsquared
    
    if not np.isnan(r2):
#         print(item_nbr, r2)
        mean+=r2
        n+=1
    
    if r2>=0.3:
        print(item_nbr,r2)
    
    results.append(model)

d:\anaconda\lib\site-packages\statsmodels\regression\linear_model.py:1386: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


37 0.344841906815
45 0.343439939064


In [45]:
mean/n

0.048168484135546336

In [52]:
no_zero = data.copy()
result=[]
stores = data['store_nbr'].unique()
print(stores)
for store in list(stores):
    item_nbr = data[(data['store_nbr']==store) & (data['units']!=0)]
    this_stores_items = item_nbr['item_nbr'].unique()
    print(store, this_stores_items)
    for each in list(this_stores_items):
        tok = no_zero[(no_zero['store_nbr']==store) & (no_zero['item_nbr']==each)]
        result.append(tok)
      
no_zero = reduce((lambda x,y : x.append(y)), result)

[ 1  2  6 38 42  3 20 28  4 24  5 10 41 44  7 13  8  9 18 23 26 31 34 11 22
 27 12 43 14 45 15 25 32 37 40 16 17 19 21 29 33 30 36 39 35]
1 [ 9 28 51 40 93 89 99 47]
2 [ 5 44 11 85 93]
6 [  9  45  14 107]
38 [ 9 45 52 93 86 98]
42 [ 5 45 82]
3 [  5  45  93   2 102 109]
20 [  5  41  74  18  93 106]
28 [ 5 45 35  2]
4 [  9  27  90  93 101  97  96]
24 [ 6 43 60 76 97]
5 [ 16  32  37  20  93 105]
10 [ 5 37 93 21 86]
41 [ 16  68  61  93  86 108]
44 [  5  68  62  93  94 105]
7 [ 5  7 23 93 95]
13 [  5  36  46  93 104]
8 [  5  23  59  93  84 100]
9 [  5  10  45  93  42 105]
18 [ 5 37 72 57]
23 [ 5 29 45 56]
26 [ 5 45  2]
31 [  9  45  93  67  98 104]
34 [ 9 31 45 52 93 84]
11 [  9  68  61  86  87 110]
22 [  9  68  93  78  54 104]
27 [ 9 68 80 21 93]
12 [16 37 79 93 86]
43 [ 16  37  73  93   3 105]
14 [ 9 50 70 25 93 88 33 15]
45 [ 9 50 34 26  4 22]
15 [ 5 17 45 49 61 93 86 15]
25 [ 9 45 81 12 50 84 93]
32 [ 9 45 58 93 19 51 86]
37 [  5  45  53  15 104 105  38]
40 [  5  45  51  55  93  15 106]


In [49]:
len(no_zero.dropna(axis=1).columns)

21

In [67]:
no_zero = no_zero.dropna(axis=1,how='any')

formula = columns_to_relation(cols,None,'units',False)


cols = list(no_zero.columns)
remove = ['date','store_nbr','item_nbr','station_nbr','codesum','day','month','year','sunrise','sunset']

for each in remove:
    cols.remove(each)

formula = columns_to_relation(cols,None,'units',False)

In [68]:
formula

'units ~ avgspeed + cool + dewpoint + heat + preciptotal + resultdir + resultspeed + tavg + tmax + tmin'

In [70]:
results=[0,]
mean = 0
n = 0
print(no_zero['item_nbr'].unique())
for item_nbr in no_zero['item_nbr'].unique():
    frame = no_zero[no_zero['item_nbr']==item_nbr]
    
    model = sm.OLS.from_formula(formula,data=frame)
    
    r2 = model.fit().rsquared
    
    if not np.isnan(r2):
        mean+=r2
        n+=1
        
        if r2>=0.3:
            print(item_nbr, r2)
    
    results.append(model)

[  9  28  51  40  93  89  99  47   5  44  11  85  45  14 107  52  86  98
  82   2 102 109  41  74  18 106  35  27  90 101  97  96   6  43  60  76
  16  32  37  20 105  21  68  61 108  62  94   7  23  95  36  46 104  59
  84 100  10  42  72  57  29  56  67  31  87 110  78  54  80  79  73   3
  50  70  25  88  33  15  34  26   4  22  17  49  81  12  58  19  53  38
  55  39  77  64  48  71  83  69  75  91   8  92  65  13  30   1 103 111
  66  63  24]


In [72]:
mean/n

0.037502832466783986

In [73]:
results=[0,]
mean = 0
n = 0
print(no_zero['store_nbr'].unique())
for store_nbr in no_zero['store_nbr'].unique():
    store = no_zero[no_zero['store_nbr']==store_nbr]
    
    store_result=[]
    
    print(store_nbr, store['item_nbr'].unique())
    
    for item_nbr in store['item_nbr'].unique():
        
        frame = store[store['item_nbr']==item_nbr]
        
        model = sm.OLS.from_formula(formula,data=frame)
        
        r2 = model.fit().rsquared
        
        if not np.isnan(r2):
            if r2>=0.3:
                print(store_nbr, item_nbr, r2)
            mean+=r2
            n+=1
        
        store_result.append(model)
        
    results.append(store_result)
    


[ 1  2  6 38 42  3 20 28  4 24  5 10 41 44  7 13  8  9 18 23 26 31 34 11 22
 27 12 43 14 45 15 25 32 37 40 16 17 19 21 29 33 30 36 39 35]
1 [ 9 28 51 40 93 89 99 47]
2 [ 5 44 11 85 93]
6 [  9  45  14 107]
38 [ 9 45 52 93 86 98]
42 [ 5 45 82]
3 [  5  45  93   2 102 109]
20 [  5  41  74  18  93 106]
28 [ 5 45 35  2]
4 [  9  27  90  93 101  97  96]
24 [ 6 43 60 76 97]
5 [ 16  32  37  20  93 105]
10 [ 5 37 93 21 86]
41 [ 16  68  61  93  86 108]
44 [  5  68  62  93  94 105]
7 [ 5  7 23 93 95]
13 [  5  36  46  93 104]
8 [  5  23  59  93  84 100]
9 [  5  10  45  93  42 105]
18 [ 5 37 72 57]
23 [ 5 29 45 56]
26 [ 5 45  2]
31 [  9  45  93  67  98 104]
34 [ 9 31 45 52 93 84]
11 [  9  68  61  86  87 110]
22 [  9  68  93  78  54 104]
27 [ 9 68 80 21 93]
12 [16 37 79 93 86]
43 [ 16  37  73  93   3 105]
14 [ 9 50 70 25 93 88 33 15]
45 [ 9 50 34 26  4 22]
15 [ 5 17 45 49 61 93 86 15]
25 [ 9 45 81 12 50 84 93]
32 [ 9 45 58 93 19 51 86]
37 [  5  45  53  15 104 105  38]
40 [  5  45  51  55  93  15 106]


In [74]:
mean/n

0.041543020092726118